# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [5]:
# --------- importando libs -------------
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# ------ Abrindo um bot Chrome ---------
navegador = webdriver.Chrome()

# ---- Pegando cotação do dolar => 'dolarcot' ---
navegador.get('https://www.google.com/')
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
dolarcot = navegador.find_element(By.XPATH,
                                  '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(f'Cotação do Dolar: R$ {dolarcot}')

# ---- Pegando cotação do Bitcoin --------------
navegador.get('https://www.google.com/')
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação bitcoin')
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
bitcot = navegador.find_element(By.XPATH, '//*[@id="crypto-updatable_2"]/div/div[2]/span[1]').text
bitcot = bitcot.replace('.','')
bitcot = bitcot.replace(',','.')
print(f'Cotação do Bitcoin: R$ {bitcot}')

# ---- Pegando cotação do Euro -----------------
navegador.get('https://www.google.com/')
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element(By.XPATH,
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
eurocot = navegador.find_element(By.XPATH,
                                  '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(f'Cotação do Euro: R$ {eurocot}')

# ---- Pegando cotação do Ouro ------------------
navegador.get('https://goldrate.com/pt-br/grama-do-ouro-preco-cotacao-valor/')
ourocot = navegador.find_element(By.XPATH, 
                                 '//*[@id="main"]/div/div/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/h2/em').text
print(f'Cotação do Ouro: R$ {ourocot}')



navegador.quit()

Cotação do Dolar: R$ 5.4666999999999994
Cotação do Bitcoin: R$ 228568.19
Cotação do Euro: R$ 6.2040999999999995
Cotação do Ouro: R$ 323.95


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [6]:
import pandas as pd

bitcot = float(bitcot)
tabela = pd.read_excel('Produtos.xlsx')
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(dolarcot)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(eurocot)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(ourocot)
display(tabela)



,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.4667,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.2041,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.4667,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.4667,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.2041,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.4667,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,323.9500,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [7]:
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.4667,5466.645333,1.40,7653.303466
1,Carro Renault,4500.00,Euro,6.2041,27918.450000,2.00,55836.900000
2,Notebook Dell,899.99,Dólar,5.4667,4919.975333,1.70,8363.958066
3,IPhone,799.00,Dólar,5.4667,4367.893300,1.70,7425.418610
4,Carro Fiat,3000.00,Euro,6.2041,18612.300000,1.90,35363.370000
5,Celular Xiaomi,480.48,Dólar,5.4667,2626.640016,2.00,5253.280032
6,Joia 20g,20.00,Ouro,323.9500,6479.000000,1.15,7450.850000


### Agora vamos exportar a nova base de preços atualizada

In [8]:
tabela.to_excel('Produtos Novo.xlsx',index=False)